In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

import ollama
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


/Users/adityasinha/Desktop/AiPrivaters/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
app = FastAPI()

In [3]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [4]:
class Article(BaseModel):
    article_hi: str
    language: int

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

try:
    tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
except OSError:
    print("Downloading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", download=True)


In [5]:
@app.post("/translate/")
async def translate(article: Article):
    if int(article.language) != 1:
        if article.language == 2:
            target_lang = "eng_Latn"
            source_lang = "hin_Deva"
        else:
            target_lang = "eng_Latn"
            source_lang = "kan_Knda"

        article_hi = article.article_hi
        translator = pipeline(task="translation", model=model, tokenizer=tokenizer, src_lang=source_lang, tgt_lang=target_lang, max_length=100)
        output = translator(article.article_hi)
        translated_text = output[0]['translation_text']
        
        answer = ollama.chat(model="mistral", stream=False, messages=[{'role': 'user', 'content': translated_text}])
        llm_output = answer["message"]["content"]
        
        translator = pipeline(task="translation", model=model, tokenizer=tokenizer, src_lang=target_lang, tgt_lang=source_lang, max_length=900)
        output = translator(llm_output)
        translated_text = output[0]['translation_text']
        return {"bot_answer": translated_text}
    else:
        article_hi = article.article_hi
        answer = ollama.chat(model="mistral", stream=False, messages=[{'role': 'user', 'content': article_hi}])
        llm_output = answer["message"]["content"]
        return {"bot_answer": llm_output}

In [6]:
!ngrok authtoken 2ei4O5VVl4qaEnZgG1YxhuzspVU_4WEAzeD6STWdUvwxx5UHe


Authtoken saved to configuration file: /Users/adityasinha/Library/Application Support/ngrok/ngrok.yml


In [7]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [11136]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://1e91-117-236-190-193.ngrok-free.app
